In [8]:
from DeepLearningToRank import tfFLAGS
tfFLAGS.LEARNING_RATE_DECAY_FACTOR = 0.1;           tfFLAGS.NUM_CLASSES = 2;       tfFLAGS.run_once = True
tfFLAGS.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000;   tfFLAGS.batch_size = 128;       tfFLAGS.use_fp16 = False
tfFLAGS.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000;    tfFLAGS.max_steps = 400;      tfFLAGS.log_frequency = 100
tfFLAGS.MOVING_AVERAGE_DECAY = 0.9999;   tfFLAGS.NUM_EPOCHS_PER_DECAY = 350.0;      tfFLAGS.IMAGE_SIZE = 10;
tfFLAGS.INITIAL_LEARNING_RATE = 0.1;     tfFLAGS.eval_interval_secs = 60 * 5;       tfFLAGS.num_examples = 1000

In [9]:
import tensorflow as tf
from MainSrc.PythonVersionHandler import *

In [10]:
conv1Size = 3; conv1Shape = [conv1Size, conv1Size, 3, 64]
conv2Size = 5; conv2Shape = [conv2Size, conv2Size, 64, 128]

pool1S = 2; pool1ksize=[1, pool1S, pool1S, 1]; pool1St = 2; pool1strides=[1, pool1St, pool1St, 1]; pool1padding='SAME'
pool2S = 2; pool2ksize=[1, pool2S, pool2S, 1]; pool2St = 2; pool2strides=[1, pool2St, pool2St, 1]; pool2padding='SAME'

local3InputDepth = 384; local3OutputDepth = 384
local4InputDepth = local3OutputDepth; local4OutputDepth = 192
softmax_linearInput = local4OutputDepth

In [11]:
print_('Network summary:')
print_('conv1Shape:', conv1Shape)
print_('pool1ksize:', pool1ksize, ' | pool1strides:', pool1strides, ' | pool1padding:', pool1padding)
print_('conv2Shape:', conv2Shape)
print_('pool2ksize:', pool2ksize, ' | pool2strides:', pool2strides, ' | pool2padding:', pool2padding)
print_('local3InputDepth:', local3InputDepth, ' | local3OutputDepth:', local3OutputDepth)
print_('local4InputDepth:', local4InputDepth, ' | local4OutputDepth:', local4OutputDepth)
print_('softmax_linearInput:', softmax_linearInput)
print_()
numParamConv1 = (conv1Shape[0] * conv1Shape[1] * conv1Shape[2] + 1) * conv1Shape[-1]
print_('Number of hidden parameters of conv1:', numParamConv1)
#print_('Number of hidden parameters of norm1:', numParamConv1 / pool1S**1)
numParamConv2 = (conv2Shape[0] * conv2Shape[1] * conv2Shape[2] + 1) * conv2Shape[-1]
print_('Number of hidden parameters of conv2:', numParamConv2)
numParamLocal3 = (local3InputDepth + 1) * local3OutputDepth
print_('Number of hidden parameters of local3:', numParamLocal3)
numParamLocal4 = (local4InputDepth + 1) * local4OutputDepth
print_('Number of hidden parameters of local4:', numParamLocal4)
numParamsoftmax = (softmax_linearInput + 1) * tfFLAGS.NUM_CLASSES
print_('Number of hidden parameters of softmax:', numParamsoftmax)
print_('Total number of hidden parameters:', numParamConv1 + numParamConv2 + numParamLocal3 + numParamLocal4 + numParamsoftmax)

Network summary:
conv1Shape: [3, 3, 3, 64]
pool1ksize: [1, 2, 2, 1]  | pool1strides: [1, 2, 2, 1]  | pool1padding: SAME
conv2Shape: [5, 5, 64, 128]
pool2ksize: [1, 2, 2, 1]  | pool2strides: [1, 2, 2, 1]  | pool2padding: SAME
local3InputDepth: 384  | local3OutputDepth: 384
local4InputDepth: 384  | local4OutputDepth: 192
softmax_linearInput: 192

Number of hidden parameters of conv1: 1792
Number of hidden parameters of conv2: 204928
Number of hidden parameters of local3: 147840
Number of hidden parameters of local4: 73920
Number of hidden parameters of softmax: 386
Total number of hidden parameters: 428866


In [12]:
from DeepLearningToRank.MyModel import * 
def inferenceOnJupyter(images):
    # local3
    with tf.variable_scope('local3') as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.reshape(images, [tfFLAGS.batch_size, -1])
        local3InputDepth = reshape.get_shape()[1].value
        fc1_weights = variable_with_weight_decay('weights', shape=[1, local3OutputDepth], stddev=0.04, wd=0.004)
        fc1_biases = variable_on_cpu('biases', [local3OutputDepth], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases, name=scope.name)        
        activation_summary(local3)

    # local4
    with tf.variable_scope('local4') as scope:
        fc2_weights = variable_with_weight_decay('weights', shape=[local3OutputDepth, local4OutputDepth], stddev=0.04, wd=0.004)
        fc2_biases = variable_on_cpu('biases', [local4OutputDepth], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, fc2_weights) + fc2_biases, name=scope.name)
        activation_summary(local4)
        
    with tf.variable_scope('softmax_linear') as scope:
        weights = variable_with_weight_decay('weights', [softmax_linearInput, tfFLAGS.NUM_CLASSES], stddev=1/192.0, wd=0.0)
        biases = variable_on_cpu('biases', [tfFLAGS.NUM_CLASSES], tf.constant_initializer(0.0))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
        activation_summary(softmax_linear)
    return softmax_linear, fc1_weights, fc1_biases, fc2_weights, fc2_biases

In [13]:
from DeepLearningToRank import MyModel
inferenceOnMyModel = MyModel.inference
MyModel.inference = inferenceOnJupyter

In [1]:
from runGG_Project import main as run
run()

########################################################################################
########################################################################################
########################################################################################
2017-05-19 19:06:08.270739: Running on UCSC:citrisdense...
/soe/cicekm/Projects/offlineData/HDFS/Day1_iphone_6_Data/all_day_iphone_6_journey_products 2757  products have been read successfully by 2017-05-19 19:06:16.928679
train instances is being generated by 2017-05-19 19:06:16.939078
2757
835692 train instances have been generated by 2017-05-19 19:08:15.949054
/soe/cicekm/Projects/offlineData/HDFS/Day1_iphone_6_Data/all_day_iphone_6_OneHot_TrainData.txt has been saved successfully by 2017-05-19 19:08:21.108444
2017-05-19 19:08:24.506710: DONE
########################################################################################
########################################################################################
###